In [ ]:
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from pyspark.sql import SparkSession

# Crie uma instância do SparkContext
sc = SparkContext()

# Crie uma instância do GlueContext
glueContext = GlueContext(sc)

# Crie uma instância do SparkSession
spark = glueContext.spark_session

# Leia os dados da tabela do Data Catalog
table_name = "seu-nome-de-tabela-no-data-catalog"
datasource = glueContext.create_dynamic_frame.from_catalog(database = "seu-nome-de-banco-de-dados-no-data-catalog", table_name = table_name)

# Converta o DynamicFrame para DataFrame
df = datasource.toDF()

# Faça o processamento conforme necessário
df_processado = df.select("coluna1", "coluna2", ...)

# Salve os resultados no novo bucket S3
output_bucket = "s3://seu-novo-bucket/"
output_path = output_bucket + "caminho-de-saida/"
df_processado.write.parquet(output_path, mode="overwrite")

In [ ]:
from pyspark.sql import SparkSession

# Inicialize a sessão do Spark
spark = SparkSession.builder \
    .appName("Exemplo PySpark") \
    .getOrCreate()

# Carregue os resultados das dimensões do Athena usando o PySpark
dim_nome_filme_df = spark.sql("SELECT * FROM dim_nome_filme")
fact_popularidade_df = spark.sql("SELECT * FROM fact_popularidade")
dim_tempo_df = spark.sql("SELECT * FROM dim_tempo")

# Especifique o caminho para o novo bucket S3 onde deseja salvar os dados
novo_bucket_s3 = "s3://seu-novo-bucket/"

# Salve os dados das dimensões no novo bucket S3
dim_nome_filme_df.write.parquet(novo_bucket_s3 + "dim_nome_filme")
fact_popularidade_df.write.parquet(novo_bucket_s3 + "fact_popularidade")
dim_tempo_df.write.parquet(novo_bucket_s3 + "dim_tempo")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max

# Inicialize a sessão do Spark
spark = SparkSession.builder \
    .appName("Exemplo PySpark") \
    .getOrCreate()

# Leia os dados das dimensões e fatos como DataFrames
dim_nome_filme_df = spark.table("dim_nome_filme")
fact_popularidade_df = spark.table("fact_popularidade")
dim_tempo_df = spark.table("dim_tempo")

# Junte as dimensões e fatos
joined_df = dim_nome_filme_df.join(fact_popularidade_df, "id_csv") \
    .join(dim_tempo_df, "id_csv") \
    .select(
        dim_nome_filme_df.tituloPincipal,
        dim_nome_filme_df.genero,
        fact_popularidade_df.notamedia,
        fact_popularidade_df.numerovotos,
        fact_popularidade_df.popularity,
        dim_tempo_df.anolancamento,
        dim_tempo_df.release_date,
        dim_nome_filme_df.overview
    )

# Realize o agrupamento
aggregated_df = joined_df.groupBy("anolancamento") \
    .agg(
        max("numerovotos").alias("maior_numerovotos"),
        max("popularity").alias("maior_popularity")
    )

# Ordene os resultados pelo ano de lançamento mais recente
result_df = aggregated_df.orderBy("anolancamento", ascending=False)

# Exiba os resultados
result_df.show()

In [ ]:
# Realize o agrupamento
aggregated_df = joined_df.groupBy("anolancamento") \
    .agg(
        max("numerovotos").alias("maior_numerovotos"),
        max("popularity").alias("maior_popularity")
    )
